In [1]:
#####multivariate normal distribution random number generation#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import itertools
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [2]:
####任意の相関行列(分散共分散行列)を作成する関数####
##任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = np.random.uniform(lower, upper, col*col)   #相関係数の乱数ベクトルを作成
    rho = np.tril(cov_vec.reshape(col, col), k=-1)   #乱数ベクトルを下三角行列化
    Sigma = (rho + rho.T) + np.diag(np.repeat(1, col))   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = np.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(col):
        if eigen_val[i] < 0:
            eigen_val[i] = np.random.uniform(eigen_lower, eigen_upper, 1)
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(np.power(np.diag(Sigma), 0.5)[:, np.newaxis], np.power(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

##相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

##分散共分散行列から相関行列に変換する関数
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [3]:
#平均ベクトルと相関行列の生成
k = 5
mu = np.repeat(0, k)
Cov = CorM(k, 0.2, 0.5, 0.05, 0.1)

In [4]:
#関数を用いて多変量正規分布から乱数を生成
hh = 100000
y1 = np.random.multivariate_normal(mu, Cov, hh)
print(Cov)
print(np.corrcoef(y1.T))

[[1.         0.25410174 0.34863321 0.2708521  0.33047558]
 [0.25410174 1.         0.27051793 0.37340619 0.44396133]
 [0.34863321 0.27051793 1.         0.25167334 0.32549119]
 [0.2708521  0.37340619 0.25167334 1.         0.42692069]
 [0.33047558 0.44396133 0.32549119 0.42692069 1.        ]]
[[1.         0.25500438 0.34837044 0.27231209 0.33011915]
 [0.25500438 1.         0.2667486  0.37698451 0.44288673]
 [0.34837044 0.2667486  1.         0.25064365 0.32394048]
 [0.27231209 0.37698451 0.25064365 1.         0.42832043]
 [0.33011915 0.44288673 0.32394048 0.42832043 1.        ]]


In [6]:
#コレスキー分解を用いて多変量正規分布からの乱数を生成
s = np.random.normal(0, 1, hh*k).reshape(hh, k)
P = np.linalg.cholesky(Cov)
y2 = np.dot(P, s.T).T
print(Cov)
print(np.corrcoef(y2.T))

[[1.         0.25410174 0.34863321 0.2708521  0.33047558]
 [0.25410174 1.         0.27051793 0.37340619 0.44396133]
 [0.34863321 0.27051793 1.         0.25167334 0.32549119]
 [0.2708521  0.37340619 0.25167334 1.         0.42692069]
 [0.33047558 0.44396133 0.32549119 0.42692069 1.        ]]
[[1.         0.25649375 0.34979819 0.27329673 0.33093978]
 [0.25649375 1.         0.27152499 0.37168153 0.44402028]
 [0.34979819 0.27152499 1.         0.25553135 0.32438994]
 [0.27329673 0.37168153 0.25553135 1.         0.42823006]
 [0.33093978 0.44402028 0.32438994 0.42823006 1.        ]]


In [10]:
#多変量正規分布の乱数を生成する関数
def rmvnorm(mu, Cov, hh, k):
    s = mu + np.random.normal(0, 1, hh*k).reshape(hh, k)
    P = np.linalg.cholesky(Cov)
    y = np.dot(P, mu.T).T
    return y

mu = np.random.normal(0, 1, hh*k).reshape(hh, k)
rmvnorm(mu, Cov, hh, k)

array([[-2.83473982e-01, -5.14228388e-01,  4.38374421e-01,
        -3.97519286e-02, -2.44471899e-01],
       [ 9.20590790e-01,  6.59419838e-01, -2.24588717e-01,
         1.65238685e-01,  7.10493504e-01],
       [ 4.16438491e-01,  5.95647398e-01, -1.47133262e+00,
        -1.01130880e+00, -2.07927312e-01],
       ...,
       [ 5.27085842e-01, -2.78023103e+00, -1.94448978e+00,
        -1.83253474e+00, -2.96286560e+00],
       [-7.37013478e-01,  2.35290221e-01,  8.25586925e-04,
        -5.16102861e-01, -2.61464334e-02],
       [-2.18898693e+00,  4.14349054e-01, -2.00377059e+00,
         4.72333178e-01,  7.15198276e-01]])